This is a practice run at a Kagglecompetition for predicting if a Titanic passegner 

In [1]:
import pandas as pd
import tensorflow as tf

In [2]:
df = pd.read_csv("data/train.csv")

In [3]:
test_set = pd.read_csv("data/test.csv")

In [4]:
df.head()
#Note - Survived 0 means died, Survived 1 means lived
#SibSp means how many siblings on the ship
#Parch means how many parents were on the ship
#embarked means C = Cherbourg, Q = Queenstown, S = Southampton

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
df = df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
df = df.dropna()

test_set = test_set.drop(['Name', 'Ticket', 'Cabin'], axis=1)

In [6]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [7]:
test_set.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,male,34.5,0,0,7.8292,Q
1,893,3,female,47.0,1,0,7.0000,S
2,894,2,male,62.0,0,0,9.6875,Q
3,895,3,male,27.0,0,0,8.6625,S
4,896,3,female,22.0,1,1,12.2875,S


In [8]:
df.columns

Index(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Embarked'],
      dtype='object')

In [9]:
cols_to_norm = ['Fare']

In [10]:
df[cols_to_norm] = df[cols_to_norm].apply(lambda x: (x-x.min()) / (x.max() - x.min()))
test_set[cols_to_norm] = test_set[cols_to_norm].apply(lambda x: (x-x.min()) / (x.max() - x.min()))

In [11]:
df['Pclass'] = pd.to_numeric(df['Pclass'])
test_set['Pclass'] = pd.to_numeric(test_set['Pclass'])

In [12]:
type(df['Pclass'][0])

numpy.int64

In [13]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,0.014151,S
1,1,1,female,38.0,1,0,0.139136,C
2,1,3,female,26.0,0,0,0.015469,S
3,1,1,female,35.0,1,0,0.103644,S
4,0,3,male,35.0,0,0,0.015713,S


In [14]:
#p_class = tf.feature_column.categorical_column_with_hash_bucket('Pclass',hash_bucket_size=3)
p_class = tf.feature_column.numeric_column('Pclass')
sex = tf.feature_column.categorical_column_with_vocabulary_list('Sex',['male','female'])
age = tf.feature_column.numeric_column('Age')
p_arch = tf.feature_column.categorical_column_with_hash_bucket('Parch',hash_bucket_size=4)
fare = tf.feature_column.numeric_column('Fare')
embarked = tf.feature_column.categorical_column_with_vocabulary_list('Embarked',['C','S','Q'])

# embedded_p_class = tf.feature_column.embedding_column(p_class,dimension=3)
bucket_age = tf.feature_column.bucketized_column(age,boundaries=[18,25,30,35,40,56])
embedded_sex = tf.feature_column.embedding_column(sex,dimension=2)
embedded_p_arch = tf.feature_column.embedding_column(p_arch,dimension=4)
embedded_embarked = tf.feature_column.embedding_column(embarked,dimension=3)

In [15]:
feat_cols = [p_class, embedded_sex, bucket_age, fare, embedded_embarked]

In [16]:
x_data = df.drop('Survived',axis=1)
predict_data = test_set

In [17]:
x_data.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,22.0,1,0,0.014151,S
1,1,female,38.0,1,0,0.139136,C
2,3,female,26.0,0,0,0.015469,S
3,1,female,35.0,1,0,0.103644,S
4,3,male,35.0,0,0,0.015713,S


In [18]:
predict_data.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,male,34.5,0,0,0.015282,Q
1,893,3,female,47.0,1,0,0.013663,S
2,894,2,male,62.0,0,0,0.018909,Q
3,895,3,male,27.0,0,0,0.016908,S
4,896,3,female,22.0,1,1,0.023984,S


In [19]:
labels = df['Survived']

In [20]:
labels.head()

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

In [21]:
from sklearn.model_selection import train_test_split

In [22]:
X_train, X_test, y_train, y_test = train_test_split(x_data, labels, test_size=.1)

In [23]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train, y=y_train,
                                                 batch_size=8,num_epochs=1000,
                                                 shuffle=True)

In [24]:
dnn_model = tf.estimator.DNNClassifier(hidden_units=[7,7,7,7,7],feature_columns=feat_cols,n_classes=2)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_model_dir': 'C:\\Users\\karab\\AppData\\Local\\Temp\\tmptw6s26e_', '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_session_config': None, '_keep_checkpoint_max': 5, '_save_summary_steps': 100, '_tf_random_seed': 1, '_save_checkpoints_steps': None}


In [25]:
dnn_model.train(input_fn=input_func,steps=20000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\karab\AppData\Local\Temp\tmptw6s26e_\model.ckpt.
INFO:tensorflow:loss = 5.19913, step = 1
INFO:tensorflow:global_step/sec: 246.197
INFO:tensorflow:loss = 4.06634, step = 101 (0.406 sec)
INFO:tensorflow:global_step/sec: 297.767
INFO:tensorflow:loss = 4.53034, step = 201 (0.336 sec)
INFO:tensorflow:global_step/sec: 270.779
INFO:tensorflow:loss = 4.49425, step = 301 (0.385 sec)
INFO:tensorflow:global_step/sec: 274.143
INFO:tensorflow:loss = 2.56412, step = 401 (0.367 sec)
INFO:tensorflow:global_step/sec: 300.009
INFO:tensorflow:loss = 1.7098, step = 501 (0.334 sec)
INFO:tensorflow:global_step/sec: 271.969
INFO:tensorflow:loss = 2.37153, step = 601 (0.350 sec)
INFO:tensorflow:global_step/sec: 289.042
INFO:tensorflow:loss = 1.66002, step = 701 (0.346 sec)
INFO:tensorflow:global_step/sec: 294.326
INFO:tensorflow:loss = 1.42769, step = 801 (0.350 sec)
INFO:tensorflow:global_step/sec: 288.031
INF

INFO:tensorflow:global_step/sec: 269.828
INFO:tensorflow:loss = 1.49479, step = 8401 (0.371 sec)
INFO:tensorflow:global_step/sec: 243.752
INFO:tensorflow:loss = 2.13959, step = 8501 (0.414 sec)
INFO:tensorflow:global_step/sec: 318.401
INFO:tensorflow:loss = 6.71551, step = 8601 (0.325 sec)
INFO:tensorflow:global_step/sec: 257.354
INFO:tensorflow:loss = 7.38539, step = 8701 (0.381 sec)
INFO:tensorflow:global_step/sec: 262.558
INFO:tensorflow:loss = 2.08088, step = 8801 (0.368 sec)
INFO:tensorflow:global_step/sec: 272.566
INFO:tensorflow:loss = 3.86575, step = 8901 (0.380 sec)
INFO:tensorflow:global_step/sec: 272.084
INFO:tensorflow:loss = 2.09793, step = 9001 (0.370 sec)
INFO:tensorflow:global_step/sec: 259.571
INFO:tensorflow:loss = 2.30973, step = 9101 (0.387 sec)
INFO:tensorflow:global_step/sec: 259.727
INFO:tensorflow:loss = 4.64056, step = 9201 (0.368 sec)
INFO:tensorflow:global_step/sec: 294.807
INFO:tensorflow:loss = 4.30131, step = 9301 (0.356 sec)
INFO:tensorflow:global_step/se

INFO:tensorflow:global_step/sec: 317.394
INFO:tensorflow:loss = 3.02828, step = 16801 (0.310 sec)
INFO:tensorflow:global_step/sec: 300.436
INFO:tensorflow:loss = 4.05487, step = 16901 (0.351 sec)
INFO:tensorflow:global_step/sec: 280.796
INFO:tensorflow:loss = 1.68593, step = 17001 (0.347 sec)
INFO:tensorflow:global_step/sec: 284.306
INFO:tensorflow:loss = 2.46569, step = 17101 (0.359 sec)
INFO:tensorflow:global_step/sec: 297.592
INFO:tensorflow:loss = 1.07591, step = 17201 (0.338 sec)
INFO:tensorflow:global_step/sec: 294.956
INFO:tensorflow:loss = 4.06147, step = 17301 (0.339 sec)
INFO:tensorflow:global_step/sec: 270.53
INFO:tensorflow:loss = 4.47714, step = 17401 (0.369 sec)
INFO:tensorflow:global_step/sec: 286.078
INFO:tensorflow:loss = 6.8355, step = 17501 (0.346 sec)
INFO:tensorflow:global_step/sec: 292.439
INFO:tensorflow:loss = 6.38481, step = 17601 (0.347 sec)
INFO:tensorflow:global_step/sec: 259.851
INFO:tensorflow:loss = 1.31759, step = 17701 (0.371 sec)
INFO:tensorflow:global

In [26]:
eval_input_func = tf.estimator.inputs.pandas_input_fn(x=X_test,y=y_test,
                                                      batch_size=10,num_epochs=1,
                                                      shuffle=False)

In [27]:
dnn_model.evaluate(eval_input_func)

INFO:tensorflow:Starting evaluation at 2018-05-30-00:09:54
INFO:tensorflow:Restoring parameters from C:\Users\karab\AppData\Local\Temp\tmptw6s26e_\model.ckpt-20000
INFO:tensorflow:Finished evaluation at 2018-05-30-00:09:56
INFO:tensorflow:Saving dict for global step 20000: accuracy = 0.791667, accuracy_baseline = 0.611111, auc = 0.854708, auc_precision_recall = 0.863478, average_loss = 0.440942, global_step = 20000, label/mean = 0.388889, loss = 3.96848, prediction/mean = 0.418428


{'accuracy': 0.79166669,
 'accuracy_baseline': 0.6111111,
 'auc': 0.85470778,
 'auc_precision_recall': 0.86347824,
 'average_loss': 0.44094211,
 'global_step': 20000,
 'label/mean': 0.3888889,
 'loss': 3.9684789,
 'prediction/mean': 0.41842839}

In [28]:
predict_input_func = tf.estimator.inputs.pandas_input_fn(x=predict_data,shuffle=False)

In [29]:
results = dnn_model.predict(predict_input_func)

In [30]:
my_pred = list(results)

INFO:tensorflow:Restoring parameters from C:\Users\karab\AppData\Local\Temp\tmptw6s26e_\model.ckpt-20000


In [31]:
my_pred

[{'class_ids': array([0], dtype=int64),
  'classes': array([b'0'], dtype=object),
  'logistic': array([ 0.11203039], dtype=float32),
  'logits': array([-2.0701673], dtype=float32),
  'probabilities': array([ 0.88796961,  0.11203039], dtype=float32)},
 {'class_ids': array([0], dtype=int64),
  'classes': array([b'0'], dtype=object),
  'logistic': array([ 0.15683649], dtype=float32),
  'logits': array([-1.68195713], dtype=float32),
  'probabilities': array([ 0.84316349,  0.15683649], dtype=float32)},
 {'class_ids': array([0], dtype=int64),
  'classes': array([b'0'], dtype=object),
  'logistic': array([ 0.14907129], dtype=float32),
  'logits': array([-1.74190366], dtype=float32),
  'probabilities': array([ 0.85092866,  0.14907129], dtype=float32)},
 {'class_ids': array([0], dtype=int64),
  'classes': array([b'0'], dtype=object),
  'logistic': array([ 0.14885163], dtype=float32),
  'logits': array([-1.74363637], dtype=float32),
  'probabilities': array([ 0.85114831,  0.14885163], dtype=floa

In [32]:
to_export =[]

for i in my_pred:
    to_export.append(i['class_ids'][0])

In [33]:
export_table = pd.DataFrame()

In [34]:
export_table['Passenger'] = predict_data['PassengerId']

In [35]:
export_table['Survived'] = to_export

In [36]:
export_table.head()

,Passenger,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1


In [37]:
export_table.to_csv('survived.csv')